## Give folder path of Dataset

In [ ]:
import os,shutil
from IPython.core.magic import register_line_cell_magic
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
global user_defined_num_classes
user_defined_num_classes=2

# Craete File Structure

In [ ]:
## Creating Required Directory
drive_path='/content/drive/MyDrive/'
main_directory='/content/drive/MyDrive/Yolo_tiny'

try:
  os.mkdir(os.path.join(main_directory))
except:
  print("Directory Already Exists !!!")
  pass

try:
  os.mkdir(os.path.join(main_directory,"input_video"))
except:
  print("Directory Already Exists !!!")
  pass

try:
  os.mkdir(os.path.join(main_directory,"output_video"))
except:
  print("Directory Already Exists !!!")
  pass

try:
  os.mkdir(os.path.join(main_directory,"models"))
except:
  print("Directory Already Exists !!!")
  pass

Directory Already Exists !!!
Directory Already Exists !!!
Directory Already Exists !!!
Directory Already Exists !!!


# Configuring cuDNN on Colab for YOLOv4



In [ ]:
os.chdir(main_directory)

!/usr/local/cuda/bin/nvcc --version    # CUDA: Let's check that Nvidia CUDA drivers are already pre-installed and which version is it.
!nvidia-smi                            # Current Available GPU information

## This cell ensures you have the correct architecture for your respective GPU; If you command is not found, look through these GPUs, find the respective
os.environ['GPU_TYPE'] = str(os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read())

def getGPUArch(argument):
  try:
    argument = argument.strip()
    # All Colab GPUs
    archTypes = {
        "Tesla V100-SXM2-16GB": "-gencode arch=compute_70,code=[sm_70,compute_70]",
        "Tesla K80": "-gencode arch=compute_37,code=sm_37",
        "Tesla T4": "-gencode arch=compute_75,code=[sm_75,compute_75]",
        "Tesla P40": "-gencode arch=compute_61,code=sm_61",
        "Tesla P4": "-gencode arch=compute_61,code=sm_61",
        "Tesla P100-PCIE-16GB": "-gencode arch=compute_60,code=sm_60"

      }
    return archTypes[argument]
  except KeyError:
    return "GPU must be added to GPU Commands"
os.environ['ARCH_VALUE'] = getGPUArch(os.environ['GPU_TYPE'])

print("GPU Type: " + os.environ['GPU_TYPE'])
print("ARCH Value: " + os.environ['ARCH_VALUE'])

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
Sat Jul 15 06:00:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |       

# Installing Darknet for YOLOv4 on Colab




In [ ]:
!pwd   # should be inside the drive/Yolo_tiny directory

/content/drive/MyDrive/Yolo_tiny


In [ ]:
## Remove if "darknet" already exists
if os.path.isdir(os.path.join(main_directory,"darknet/"))==True:
  shutil.rmtree(os.path.join(main_directory,"darknet"))


## Clone Darknet files from Github
!git clone https://github.com/AlexeyAB/darknet.git


## Install environment from 'Makefile'
os.chdir(os.path.join(main_directory,"darknet"))     # change directory to '/content/drive/MyDrive/Yolo_tiny/darknet'
"""
Note:
      If you are on Colab Pro this works on a P100 GPU
      If you are on Colab free, you may need to change the Makefile for the K80 GPU
      this goes for any GPU, you need to change the Makefile to inform darknet which GPU you are running on.
"""
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i "s/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= ${ARCH_VALUE}/g" Makefile
!make


## Download newly released yolov4-tiny weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

Cloning into 'darknet'...
remote: Enumerating objects: 15530, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 15530 (delta 6), reused 10 (delta 4), pack-reused 15514
Receiving objects: 100% (15530/15530), 14.21 MiB | 14.31 MiB/s, done.
Resolving deltas: 100% (10418/10418), done.
Updating files: 100% (2058/2058), done.
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wun

# Set up Custom Dataset for YOLOv4

In [ ]:
%mkdir data/obj

# ## Copy train,test,valid zip files & unzip
%cp /content/drive/MyDrive/Yolo_tiny/train.zip /content
%cp /content/drive/MyDrive/Yolo_tiny/valid.zip /content
#%cp /content/drive/MyDrive/Yolo_tiny/test.zip /content
%cp /content/drive/MyDrive/Yolo_tiny/obj.names data/  # copy obj.names

!unzip /content/train.zip -d /content
!unzip /content/valid.zip -d /content
#!unzip /content/test.zip -d data


## Set up training file directories for custom dataset (copy image and labels)
%cp /content/train/*.jpg data/obj/
%cp /content/valid/*.jpg data/obj/
%cp /content/train/*.txt data/obj/
%cp /content/valid/*.txt data/obj/


with open('data/obj.data', 'w') as out:
  out.write('classes = 1\n')
  out.write('train = data/train.txt\n')
  out.write('valid = data/valid.txt\n')
  out.write('names = data/obj.names\n')
  out.write('backup = /content/drive/MyDrive/Yolo_tiny/models/')

## write train file (just the image list)
with open('data/train.txt', 'w') as out:
  for img in [f for f in os.listdir('/content/train') if f.endswith('jpg')]:
    out.write('data/obj/' + img + '\n')

## write the valid file (just the image list)
with open('data/valid.txt', 'w') as out:
  for img in [f for f in os.listdir('/content/valid') if f.endswith('jpg')]:
    out.write('data/obj/' + img + '\n')

Archive:  /content/train.zip
  inflating: /content/train/0.jpg    
  inflating: /content/train/0.txt    
  inflating: /content/train/1.jpg    
  inflating: /content/train/1.txt    
  inflating: /content/train/10.jpg   
  inflating: /content/train/10.txt   
  inflating: /content/train/100.jpg  
  inflating: /content/train/100.txt  
  inflating: /content/train/101.jpg  
  inflating: /content/train/101.txt  
  inflating: /content/train/102.jpg  
  inflating: /content/train/102.txt  
  inflating: /content/train/104.jpg  
  inflating: /content/train/104.txt  
  inflating: /content/train/105.jpg  
  inflating: /content/train/105.txt  
  inflating: /content/train/106.jpg  
  inflating: /content/train/106.txt  
  inflating: /content/train/107.jpg  
  inflating: /content/train/107.txt  
  inflating: /content/train/108.jpg  
  inflating: /content/train/108.txt  
  inflating: /content/train/109.jpg  
  inflating: /content/train/109.txt  
  inflating: /content/train/11.jpg   
  inflating: /content

# Write Custom Training Config for YOLOv4

Note for Parameter change in config file:

- **batch = 64** and **subdivisions = 16**
 If you run into any issues then up subdivisions to 32.

- width = 416,height = 416
**(these can be any multiple of 32, 416 is standard, you can sometimes improve results by making value larger like 608 but will slow down training)**

- max_batches = (# of classes) * 2000  (Min max_batch will be 6000)

- steps = (80% of max_batches), (90% of max_batches)
**(so if your max_batches = 10000, then steps = 8000, 9000)**

- filters = (# of classes + 5) * 3
**(so if you are training for one class then your filters = 18, but if you are training for 4 classes then your filters = 27)**


**Optional:** If you run into memory issues or find the training taking a super long time. In each of the three yolo layers in the cfg, change one line from random = 1 to **random = 0** to speed up training but slightly reduce accuracy of model. Will also help save memory if you run into any memory issues.

In [ ]:
## we build iteratively from base config files. This is the same file shape as cfg/yolo-obj.cfg
"""
Note:
  change line max_batches to (classes*2000 but not less than number of training images, and not less than 6000), f.e. max_batches=6000 if you train for 3 classes
  change line steps to 80% and 90% of max_batches, f.e. steps=4800,5400

"""
def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes=user_defined_num_classes                              # Number of classes
if num_classes*2000>=6000:
  max_batches = num_classes*2000
else:
  max_batches = 6000
steps1=.8*max_batches
steps2=.9*max_batches
steps_str=str(steps1)+','+str(steps2)
num_filters=(num_classes+5)*3
print("writing config for a custom YOLOv4 detector detecting number of classes: " + str(num_classes))


if os.path.exists('./cfg/custom-yolov4-tiny-detector.cfg'): os.remove('./cfg/custom-yolov4-tiny-detector.cfg')
@register_line_cell_magic        # customize iPython writefile so we can write variables
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))



print("class:",num_classes)
print("max batches:",max_batches)
print("Steps:",steps1,steps2)

writing config for a custom YOLOv4 detector detecting number of classes: 2
class: 2
max batches: 6000
Steps: 4800.0 5400.0


In [ ]:
%%writetemplate ./cfg/custom-yolov4-tiny-detector.cfg
[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=24
width=512
height=512
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.00261
burn_in=1000
max_batches = 12000
policy=steps
steps={steps_str}
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

##################################

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={num_filters}
activation=linear



[yolo]
mask = 3,4,5
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes={num_classes}
num=6
jitter=.3
scale_x_y = 1.05
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
ignore_thresh = .7
truth_thresh = 1
random=0
nms_kind=greedynms
beta_nms=0.6

[route]
layers = -4

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = -1, 23

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={num_filters}
activation=linear

[yolo]
mask = 1,2,3
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes={num_classes}
num=6
jitter=.3
scale_x_y = 1.05
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
ignore_thresh = .7
truth_thresh = 1
random=0
nms_kind=greedynms
beta_nms=0.6


%cat cfg/custom-yolov4-tiny-detector.cfg

# Train model seamlessly
press & hold (CTRL + SHIFT + i)

```
function ClickConnect(){
console.log("Working");
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click()
}
setInterval(ClickConnect,60000)
```

# Train Custom YOLOv4 Detector

In [ ]:
#!./darknet detector --train-data --configfile --pretrained_weight --flags
!./darknet detector train data/obj.data cfg/custom-yolov4-tiny-detector.cfg yolov4-tiny.conv.29 -dont_show -map

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.859038), count: 2, class_loss = 0.000569, iou_loss = 7.976305, total_loss = 7.976874 
 total_bbox = 240494, rewritten_bbox = 0.000000 % 

 (next mAP calculation at 2100 iterations) 
 Last accuracy mAP@0.50 = 100.00 %, best = 100.00 % ]2;2091/12000: loss=0.0 map=1.00 best=1.00 hours left=2.5
 2091: 0.047765, 0.047778 avg loss, 0.002610 rate, 0.818647 seconds, 100368 images, 2.502099 hours left
Loaded: 0.000038 seconds
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.932811), count: 2, class_loss = 0.000298, iou_loss = 0.402317, total_loss = 0.402615 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.814164), count: 2, class_loss = 0.001470, iou_loss = 4.061925, total_loss = 4.063395 
 total_bbox = 240498, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.0

## Resume the Training


In [ ]:
## All Available trained weights
!ls /content/drive/MyDrive/Yolo_tiny/models

custom-yolov4-tiny-detector_best.weights
custom-yolov4-tiny-detector_last.weights


In [ ]:
## kick off training from where it last saved
#!./darknet detector  --train-data  --config-file  --weights --flags


!./darknet detector train data/obj.data cfg/custom-yolov4-tiny-detector.cfg /content/drive/MyDrive/Yolo_tiny/models/custom-yolov4-tiny-detector_last.weights -dont_show -map

 CUDA-version: 11080 (12000), cuDNN: 8.9.0, GPU count: 1  
 OpenCV version: 4.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 24, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    512 x 512 x   3 ->  256 x 256 x  32 0.113 BF
   1 conv     64       3 x 3/ 2    256 x 256 x  32 ->  128 x 128 x  64 0.604 BF
   2 conv     64       3 x 3/ 1    128 x 128 x  64 ->  128 x 128 x  64 1.208 BF
   3 route  2 		                       1/2 ->  128 x 128 x  32 
   4 conv     32       3 x 3/ 1    128 x 128 x  32 ->  128 x 128 x  32 0.302 BF
   5 conv     32       3 x 3/ 1    128 x 128 x  32 ->  128 x 128 x  32 0.302 BF
   6 route  5 4 	                           ->  128 x 128 x  64 
   7 conv     64       1 x 1/ 1    128 x 128 x  64 ->  128 x 128 x  64 0.134 BF

# Infer Custom Objects with Saved YOLOv4 Weights

In [ ]:
#define utility function
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

## list out all trained models
  !ls /content/drive/MyDrive/Yolo_tiny/models


In [ ]:
## coco.names is hardcoded somewhere in the detector
# %cp data/obj.names data/coco.names

In [ ]:
#/test has images that we can test our detector on
test_images = [f for f in os.listdir('data/test/') if f.endswith('.jpg')]
import random
img_path = "data/test/" + random.choice(test_images);
print('================',img_path)


#test out our detector!
!./darknet detect cfg/custom-yolov4-tiny-detector.cfg /content/drive/MyDrive/yolo/models/custom-yolov4-tiny-detector_best.weights {img_path} -dont-show -thresh 0.50
#imShow('/content/darknet/predictions.jpg')

# Testing on Video

In [ ]:
## List all Available Input Videos
!ls /content/drive/MyDrive/Yolo_tiny/input_video

In [ ]:
input_video_path = "/content/drive/MyDrive/yolo/input_video/cleaning.mp4"
output_video_path = "/content/drive/MyDrive/yolo/output_video/cleaning.mp4"

In [ ]:
!./darknet detector demo data/obj.data cfg/custom-yolov4-tiny-detector.cfg /content/drive/MyDrive/yolo/models/custom-yolov4-tiny-detector_best.weights $input_video_path -i 0 -out_filename $output_video_path -dont_show -map

#### @Abhijit